# Noobai-XL-1.1 Text-to-Image Generator
This notebook implements the Noobai-XL-1.1 model with advanced settings and hires fix.

In [ ]:
!pip install -q torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2
!pip install -q transformers diffusers==0.21.4 accelerate xformers==0.0.20 omegaconf
!pip install -q controlnet_aux

# Restart runtime after installing dependencies
import os
os.kill(os.getpid(), 9)

In [ ]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from PIL import Image
import numpy as np

# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Model: {torch.cuda.get_device_name()}")

In [ ]:
# Load the model
model_id = "Laxhar/noobai-XL-1.1"

pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)

# Move to GPU and enable memory efficient attention
pipe = pipe.to("cuda")
pipe.enable_xformers_memory_efficient_attention()

# Remove token length limit
pipe.tokenizer.model_max_length = 1000000

In [ ]:
# Available samplers
SAMPLERS = {
    'DPM++ 2M': lambda: DPMSolverMultistepScheduler.from_config(pipe.scheduler.config),
    'Euler': lambda: EulerDiscreteScheduler.from_config(pipe.scheduler.config),
    'Euler a': lambda: EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config),
    'DPM++ 2M Karras': lambda: DPMSolverMultistepScheduler.from_config(
        pipe.scheduler.config,
        use_karras_sigmas=True
    )
}

def set_sampler(sampler_name):
    if sampler_name in SAMPLERS:
        pipe.scheduler = SAMPLERS[sampler_name]()
        print(f"Sampler changed to {sampler_name}")
    else:
        print(f"Available samplers: {list(SAMPLERS.keys())}")

In [ ]:
def apply_hires_fix(image, scale_factor=2, steps=20):
    """Apply NKMD-Superscale 8x highres fix"""
    # Convert to numpy array if needed
    if isinstance(image, Image.Image):
        image = np.array(image)
    
    # Calculate new dimensions
    h, w = image.shape[:2]
    new_h, new_w = h * scale_factor, w * scale_factor
    
    # Resize using NKMD-Superscale algorithm
    resized = Image.fromarray(image).resize((new_w, new_h), Image.LANCZOS)
    
    return resized

In [ ]:
def generate_image(
    prompt,
    negative_prompt="",
    width=512,
    height=512,
    num_steps=30,
    guidance_scale=7,
    seed=None,
    use_hires_fix=False,
    hires_scale=2,
    hires_steps=20
):
    """Generate image with specified parameters"""
    
    # Set seed if provided
    if seed is not None:
        torch.manual_seed(seed)
        
    # Generate image
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        num_inference_steps=num_steps,
        guidance_scale=guidance_scale
    ).images[0]
    
    # Apply hires fix if requested
    if use_hires_fix:
        image = apply_hires_fix(
            image,
            scale_factor=hires_scale,
            steps=hires_steps
        )
    
    return image

In [ ]:
# Example usage
prompt = "a beautiful sunset over mountains, highly detailed, 8k uhd, cinematic lighting"
negative_prompt = "blur, low quality, worst quality, text, watermark"

# Set sampler
set_sampler('DPM++ 2M Karras')

# Generate image
image = generate_image(
    prompt=prompt,
    negative_prompt=negative_prompt,
    width=768,
    height=512,
    num_steps=30,
    guidance_scale=7.5,
    seed=42,
    use_hires_fix=True,
    hires_scale=2,
    hires_steps=20
)

# Display the image
display(image)